In [53]:
# import all libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import re
from sqlalchemy import create_engine # needed for DB connection.
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import scale
from sklearn.feature_selection import RFE, SelectFromModel
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier

In [54]:
#read csv file and remove data with null values
df = pd.read_csv("surgical_case_durations.csv", sep=";", encoding="ISO-8859-1")
df.dropna(subset=['Benadering', 'Geslacht', 'AF', 'CCS', 'NYHA', 'Euroscore1'], how='all', inplace=True)
df.dropna(subset=['Operatieduur'], how='all', inplace=True)
print(df.shape)
df.head()

(3530, 36)


,Operatietype,Chirurg,Anesthesioloog,Benadering,OK,Casustype,Dagdeel,Leeftijd,Geslacht,AF,...,Hypertensie,Perifeer vaatlijden,CCS,NYHA,Aantal anastomosen,HLM,Geplande operatieduur,Operatieduur,Ziekenhuis ligduur,IC ligduur
2,Ascendensvervanging,"4,00","8,00",Volledige sternotomie,TOK1,Electief,Ochtend,78.0,V,N,...,N,J,0.0,2.0,0.0,N,229.0,170.0,"4,00","2,00"
3,Ascendensvervanging,"7,00","6,00",Volledige sternotomie,TOK2,Electief,Middag,66.0,V,J,...,N,N,2.0,1.0,0.0,N,140.0,190.0,"6,00","1,00"
5,Ascendensvervanging,"1,00","7,00",Volledige sternotomie,TOK1,Spoed,Ochtend,79.0,M,N,...,N,N,4.0,4.0,0.0,N,180.0,184.0,"9,00","0,00"
6,Ascendensvervanging,"4,00","10,00",Volledige sternotomie,TOK3,Electief,Ochtend,71.0,M,N,...,J,N,0.0,1.0,0.0,N,241.0,239.0,"6,00","1,00"
7,Ascendensvervanging,"3,00","15,00",Volledige sternotomie,TOK2,Electief,Ochtend,66.0,V,N,...,J,N,0.0,1.0,0.0,J,240.0,269.0,"11,00","0,00"


In [55]:
#feature selection
df = df[["Operatietype", "Chirurg", "Anesthesioloog", "Benadering", "OK", "Casustype", "Dagdeel",
             "Aantal anastomosen", "HLM", "Geplande operatieduur", "Operatieduur", "Ziekenhuis ligduur", "IC ligduur",
             "Leeftijd", "Geslacht", "AF", "Chronische longziekte", "Extracardiale vaatpathie",
             "Actieve endocarditis", "Hypertensie", "Pulmonale hypertensie", "Slechte mobiliteit", "Hypercholesterolemie",
             "Perifeer vaatlijden", "BMI", "DM", "Eerdere hartchirurgie","Linker ventrikel functie", "Nierfunctie",
            "Kritische preoperatieve status", "Myocard infact <90 dagen", "Aorta chirurgie", "Euroscore1", "Euroscore2"]]
#rename collumns to English
df = df.rename(columns={"Operatietype": "surgery_type", "Chirurg": "surgeon", "Benadering": "surgical_approach",
                        "Anesthesioloog": "anesthesiologist", "OK": "operation_room", "Casustype": "urgency",
                        "Dagdeel": "time_of_day", "Aantal anastomosen": "bypasses_amount", "HLM" : "cardiopulmonary_bypass_use",
                        "Geplande operatieduur" : "planned_surgery_duration", "Operatieduur" : "surgery_duration",
                        "Ziekenhuis ligduur":"hospital_days", "IC ligduur":"IC_days", "Leeftijd" : "patient_age", 
                        "Geslacht": "patient_gender", "AF": "atrial_fibrillation", "Chronische longziekte": "chronic_lung_disease",
                        "Extracardiale vaatpathie": "extracardial_arteriopathy", "Actieve endocarditis":"active_endocarditis",
                        "Hypertensie" : "hypertension", "Pulmonale hypertensie": "pulmonary_hypertension",
                        "Slechte mobiliteit" : "poor_mobility", "Hypercholesterolemie": "hypercholesterolemia", 
                        "Perifeer vaatlijden": "peripherial_vascular_disease", "DM" : "diabetes_mellitus", 
                        "Eerdere hartchirurgie": "previous_heart_surgery", "Kritische preoperatieve status": "critical_pre_or_state", 
                        "Myocard infact <90 dagen": "mycordial_infarction", "Aorta chirurgie": "aortic_surgery",
                       "Linker ventrikel functie": "left_ventricle", "Nierfunctie": "renal_function"})
#drop duplicate data
df = df.drop_duplicates(ignore_index=True, keep="last")
#surgery["surgeryid"] = surgery.reset_index().index

In [56]:
#translate the categories in surgical type and impute any categories with count < 20 and NULL to Other types
value_counts = df['surgery_type'].value_counts()

# Create a mapping of values to change
values_to_change = value_counts[value_counts < 20].index

# Use the apply method to change values based on the condition
df['surgery_type'] = df['surgery_type'].apply(lambda x: 'Other types' if x in values_to_change else x)
df['surgery_type'] = df['surgery_type'].replace({'Lobectomie of segmentresectie':'Lobectomy or segment resection',
                                                 'Mediastinoscopie' : 'Mediastinoscopy',
                                                 'VATS Boxlaesie' : 'video assisted thoracic surgery'})
#impute null value
df['surgery_type'].fillna('Other types', inplace=True)

In [57]:
#translate the categories in surgical approach and impute Parasternaal links,
#Parasternaal rechts, Dwarse sternotomie, Xiphoidaal, and NULL to Other approaches
df['surgical_approach'] = df['surgical_approach'].replace({'Volledige sternotomie':'Full sternotomy',
                                                           'Antero lateraal links' : 'Left antero lateral',
                                                           'Antero lateraal rechts' : 'Right antero lateral',
                                                           'Postero lateraal links' : 'Left postero lateral',
                                                           'Postero lateraal rechts' : 'Right postero lateral',
                                                           'Partiële sternotomie' : 'Partial sternotomy',
                                                           'Dwarse sternotomie' : 'Other approaches',
                                                           'Parasternaal rechts' : 'Other approaches',
                                                           'Parasternaal links' : 'Other approaches',
                                                           'Xiphoidaal' : 'Other approaches'})
df['surgical_approach'].fillna('Other approaches', inplace=True)

In [58]:
#translate the categories in urgency and impute NULL value to unknown
df['urgency'] = df['urgency'].replace({'Electief':'Elective', 
                                       'Acuut < 30 minuten' : 'Acute (< 30 minutes)',
                                       'Spoed' : 'Acute',
                                       'Spoed < 24 uur' : 'Emergency (< 24 hours)',
                                       'Spoed < 5 uur' : 'Acute (< 5 hours)'})
df['urgency'].fillna('Unknown', inplace=True)

#translate the categories in time_of_day & pulmonary_hypertension
df['time_of_day'] = df['time_of_day'].replace({'Ochtend':'Morning',
                                               'Middag' : 'Afternoon',
                                               'Nacht' : 'Evening and night',
                                               'Avond' : 'Evening and night',})

df['pulmonary_hypertension'] = df['pulmonary_hypertension'].replace({'Normaal':'Normal',
                                                                     'Matig' : 'Moderate',
                                                                     'Ernstig' : 'Severe'})


In [59]:
#impute NULL value in bypasses_amount to 0 and alter the type to int
df['bypasses_amount'].fillna('0.00', inplace=True)
df['bypasses_amount'] = df['bypasses_amount'].astype(float)
df['bypasses_amount'] = df['bypasses_amount'].astype(int)

#create new collumn called late status to check the status of planned_surgery_duration and check the time difference
df["latestatus"] = np.where(df["surgery_duration"]>df["planned_surgery_duration"], "Late", "Not Late")
df["time_difference"] = abs(df["surgery_duration"]-df["planned_surgery_duration"])
conditions  = [ df["time_difference"] >= 120, (df["time_difference"] < 120) & (df["time_difference"]>= 60), 
               (df["time_difference"] < 60) & (df["time_difference"]>= 30), (df["time_difference"]< 30)  ]
choices     = [ "more than 2 hours", 'more than 1 hour', 'more than 30 minutes', 'less than 30 minutes' ]
df["time_difference"] = np.select(conditions, choices, default=np.nan)


In [60]:
#translate collumns' categories
df['cardiopulmonary_bypass_use'] = df['cardiopulmonary_bypass_use'].replace({'J':'Y'})
df['patient_gender'] = df['patient_gender'].replace({'V':'F'})
df['atrial_fibrillation'] = df['atrial_fibrillation'].replace({'J':'Y'})
df['chronic_lung_disease'] = df['chronic_lung_disease'].replace({'J':'Y'})
df['active_endocarditis'] = df['active_endocarditis'].replace({'J':'Y'})
df['poor_mobility'] = df['poor_mobility'].replace({'J':'Y'})
df['hypertension'] = df['hypertension'].replace({'J':'Y'})
df['diabetes_mellitus'] = df['diabetes_mellitus'].replace({'J':'Y'})
df['peripherial_vascular_disease'] = df['peripherial_vascular_disease'].replace({'J':'Y'})
df['hypercholesterolemia'] = df['hypercholesterolemia'].replace({'J':'Y'})
df['extracardial_arteriopathy'] = df['extracardial_arteriopathy'].replace({'J':'Y'})
df['critical_pre_or_state'] = df['critical_pre_or_state'].replace({'J':'Y'})
df['aortic_surgery'] = df['aortic_surgery'].replace({'J':'Y'})
df['mycordial_infarction'] = df['mycordial_infarction'].replace({'J':'Y'})

#alter Euroscore1's type to float
df['Euroscore1'] = df['Euroscore1'].str.replace(',', '.').astype(float)
df['Euroscore2'] = df['Euroscore2'].str.replace(',', '.').astype(float)
df['Euroscore2'] = df['Euroscore2'].fillna(df['Euroscore2'].mean())

#impute NULL value in patient_age to mean
df['patient_age']=df['patient_age'].fillna(df['patient_age'].mean())

In [61]:
df['left_ventricle'] = df['left_ventricle'].replace({'Goed':'Good',
                                                               'Matig' : 'Moderate',
                                                               'Slecht' : 'Poor',
                                                               'Heel Slecht' : 'Very Poor'})
df['renal_function'] = df['renal_function'].replace({'Normaal':'Normal',
                                                               'Matig' : 'Moderate',
                                                               'Slecht' : 'Poor'})

df['left_ventricle']=df['left_ventricle'].fillna('Good')
df['renal_function']=df['renal_function'].fillna('Normal')

In [ ]:
#create surgery dataframe
surgery = df[["surgery_type", "surgeon", "anesthesiologist", "surgical_approach", "operation_room", "urgency", "time_of_day",
             "bypasses_amount", "cardiopulmonary_bypass_use", "planned_surgery_duration", "surgery_duration", "hospital_days",
              "IC_days"]]
#set surgeryid
surgery["surgeryid"] = surgery.reset_index().index
surgery = surgery[["surgeryid", "surgery_type", "surgeon", "surgical_approach", "anesthesiologist", "operation_room",
                   "urgency", "time_of_day", "bypasses_amount", "cardiopulmonary_bypass_use"]]

In [ ]:
#create duration dataframe
duration = df[["planned_surgery_duration","surgery_duration","hospital_days","IC_days", "latestatus", "time_difference"]]
duration["surgeryid"] = duration.reset_index().index
duration = duration[["surgeryid", "planned_surgery_duration","surgery_duration","hospital_days","IC_days", "latestatus", "time_difference"]]

In [ ]:
#create patient dataframe
patient = df[["patient_age", "patient_gender", "atrial_fibrillation", "chronic_lung_disease", "extracardial_arteriopathy",
             "active_endocarditis", "hypertension", "pulmonary_hypertension", "poor_mobility", "hypercholesterolemia",
             "peripherial_vascular_disease", "BMI", "diabetes_mellitus", "previous_heart_surgery",
            "critical_pre_or_state", "mycordial_infarction", "aortic_surgery", "Euroscore1"]]
patient["patientid"] = patient.reset_index().index

In [25]:
#create link to database
driver="postgresql"
username="dab_ds23241a_53"
dbname=username
password="pxMvPSGXWIpHDHdL"
server="bronto.ewi.utwente.nl"
port="5432"
# Creating the connetcion pool for SQL
engine = create_engine(driver+"://"+username+":"+password+"@"+server+":"+port+"/"+dbname)
surgery.to_sql('surgery', engine, schema='project', index=False, if_exists='replace')
duration.to_sql('duration', engine, schema='project', index=False, if_exists='replace')
patient.to_sql('patient', engine, schema='project', index=False, if_exists='replace')

530

In [62]:
#create input and output for Data Mining
X = df.loc[:, ["surgery_type", "surgeon", "anesthesiologist", "surgical_approach", "operation_room", "urgency", "time_of_day",
                    "bypasses_amount", "cardiopulmonary_bypass_use","patient_age","left_ventricle", "renal_function", "patient_gender", "atrial_fibrillation", 
                    "chronic_lung_disease", "extracardial_arteriopathy", "active_endocarditis", "hypertension", 
                    "pulmonary_hypertension", "poor_mobility", "hypercholesterolemia", "peripherial_vascular_disease", 
                    "diabetes_mellitus", "previous_heart_surgery", "critical_pre_or_state", "mycordial_infarction", 
                    "Euroscore1", "Euroscore2"]]
y = df['surgery_duration']

In [63]:
# creating dummy variables for categorical variables
surgery_categorical = X.select_dtypes(include=['object'])
surgery_categorical.head()


# convert into dummies
surgery_dummies = pd.get_dummies(surgery_categorical, drop_first=True)
surgery_dummies.head()


# drop categorical variables 
X = X.drop(list(surgery_categorical.columns), axis=1)


# concat dummy variables with X
X = pd.concat([X, surgery_dummies], axis=1)


# rescale the features
cols = X.columns
X = pd.DataFrame(scale(X))
X.columns = cols


# split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.7,
                                                    test_size = 0.3, random_state=40)

In [64]:
# creating a KFold object with 5 splits 
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)

# specify range of hyperparameters
hyper_params = [{'n_features_to_select': list(range(2, 96))}]

# specify model
lm = LinearRegression()
lm.fit(X_train, y_train)
rfe = RFE(lm)
# set up GridSearchCV()
model_cv = GridSearchCV(estimator = rfe, 
                        param_grid = hyper_params, 
                        scoring= 'neg_root_mean_squared_error', 
                        cv = folds, 
                        verbose = 1,
                        return_train_score=True)
# fit the model
model_cv.fit(X_train, y_train) 

Fitting 5 folds for each of 94 candidates, totalling 470 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=100, shuffle=True),
             estimator=RFE(estimator=LinearRegression()),
             param_grid=[{'n_features_to_select': [2, 3, 4, 5, 6, 7, 8, 9, 10,
                                                   11, 12, 13, 14, 15, 16, 17,
                                                   18, 19, 20, 21, 22, 23, 24,
                                                   25, 26, 27, 28, 29, 30, 31, ...]}],
             return_train_score=True, scoring='neg_root_mean_squared_error',
             verbose=1)

In [68]:
# test the model - linear regression
y_pred = model_cv.predict(X_test)
#calculate r2, rmse, and mae of the model
r2 = r2_score(y_test, y_pred)
print("r2:", r2)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
mae = mean_absolute_error(y_test, y_pred)
print("MAE:", mae)

r2: 0.3812475677106034
RMSE: 66.84968347593991
MAE: 46.24035827960299


In [33]:
#calculate r2, rmse, and mae of the existing prediction
r2 = r2_score(df["surgery_duration"], df["planned_surgery_duration"])
print("r2:", r2)
mse = mean_squared_error(df["surgery_duration"], df["planned_surgery_duration"])
rmse = np.sqrt(mse)
print("RMSE:", rmse)
mae = mean_absolute_error(df["surgery_duration"], df["planned_surgery_duration"])
print("MAE:", mae)

r2: 0.11795274149678647
RMSE: 80.08599839428986
MAE: 53.994334277620396


In [66]:
# test the model - random forest
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100,random_state=0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
#calculate r2, rmse, and mae of the model
r2 = r2_score(y_test, y_pred)
print("r2:", r2)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
mae = mean_absolute_error(y_test, y_pred)
print("MAE:", mae)

r2: 0.3603814848696397
RMSE: 67.9675177249464
MAE: 46.200199874095055


In [67]:
from sklearn.svm import SVR
model = SVR(kernel='linear', C=1.0, epsilon=0.2)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
#calculate r2, rmse, and mae of the model
r2 = r2_score(y_test, y_pred)
print("r2:", r2)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
mae = mean_absolute_error(y_test, y_pred)
print("MAE:", mae)

r2: 0.3755976774581339
RMSE: 67.15419549112525
MAE: 44.295712278532676
